### Imports

In [61]:
import numpy as np
from numpy import array, log, exp, where, vectorize, arange
from bond_pricing import bond_duration, duration, bond_price, edate
from isda_daycounters import actual360, actual365, actualactual, thirty360, thirtyE360, thirtyE360ISDA
from datetime import datetime

### Standard Inputs

In [124]:
coupon = 0.032
par = 100
ytm = 0.032
settle = datetime(2027, 6, 11)
mat = datetime(2030, 10, 15)
freq = 2
daycount = 'actual/360'

### Approximate Modified & Macaulay Duration

Approximate Modified Duration Calculation:
$$\text{Approx. Ann. Mod. Duration}=\frac{\left(PV_{-}\right)-\left(PV_{+}\right)}{2\times\left(\Delta \text{Yield}\right)\times\left(PV_0\right)}$$

Approximate Macaulay Duration
$$\text{Ann. Mac. Duration}=\text{Approx. Ann. Mod. Duration}\times\left(1+r\right)$$

In [55]:
def approx_duration(settle, cpn, mat, yld, freq, face, daycount, delta_yld, modified=True):
    ytm_low = ytm - delta_yld
    ytm_high = ytm + delta_yld
    price = bond_price(settle=settle, cpn=cpn, mat=mat, yld=yld, freq=freq, face=face, daycount=daycount)
    price_high = bond_price(settle=settle, cpn=cpn, mat=mat, yld=ytm_low, freq=freq, face=face, daycount=daycount)
    price_low = bond_price(settle=settle, cpn=cpn, mat=mat, yld=ytm_high, freq=freq, face=face, daycount=daycount)
    
    duration = (price_high - price_low) / (2 * delta_yld * price)
    
    if modified:
        return duration
    else:
        return duration * (1 + yld/freq)

delta = 0.0005
modified = approx_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, daycount=daycount, delta_yld=delta, modified=True)
macaulay = approx_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, daycount=daycount, delta_yld=delta, modified=False)
print(f'{modified = :0.4f}')
print(f'{macaulay = :0.4f}')


modified = 4.5868
macaulay = 4.6602


### Macaulay Duration

$$MacDur=\left\lbrace\frac{1+r}{r}-\frac{1+r+\left\lbrack N\times\left(c-r\right)\right\rbrack}{c\times\left\lbrack\left(1+r\right)^{N}-1+r\right\rbrack}\right\rbrace-\frac{t}{T}$$

- *r* is the yield-to-maturity per period;
- *N* is the number of evenly spaced periods to maturity as of the beginning of the current period;
- *c* is the coupon rate per period;
- *t* is the number of days from the last coupon payment to the settlement date; and
- *T* is the number of days in the coupon period.

In [125]:
print(bond_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, modified=False, daycount=daycount))

3.180584640456848


| Period | Time to Reciept | Cash Flow | PV       | Weight       | Time x Weight |
| ------ | --------------- | --------- | -------- | ------------ | ------------- |
| 1      | 1               | 1.6       | 1.5748   | 0.0157       | 0.0157        |
| 2      | 2               | 1.6       | 1.5500   | 0.0155       | 0.0310        |
| 3      | 3               | 1.6       | 1.5256   | 0.0153       | 0.0458        |
| 4      | 4               | 1.6       | 1.5016   | 0.0150       | 0.0601        |
| 5      | 5               | 1.6       | 1.4779   | 0.0148       | 0.0739        |
| 6      | 6               | 1.6       | 1.4546   | 0.0145       | 0.0873        |
| 7      | 7               | 1.6       | 1.4317   | 0.0143       | 0.1002        |
| 8      | 8               | 1.6       | 1.4092   | 0.0141       | 0.1127        |
| 9      | 9               | 1.6       | 1.3870   | 0.0139       | 0.1248        |
| 10     | 10              | 101.6     | 86.6875  | 0.8669       | 8.6688        |
|        |                 |           | 100.0000 | 1.0000       | 9.3203        |
|        |                 |           |          | Mac Duration | 4.6601        |
|        |                 |           |          | Mod Duration | 4.5868        |

In [132]:
from prettytable import PrettyTable

# approximate number of full coupon periods left
n = int(freq * (mat - settle).days / 360)
# the divisor of 360 guarantees this is an overestimate
# we keep reducing n till it is right
while (edate(mat, -n * 12 / freq) <= settle):
    n -= 1
next_coupon = edate(mat, -n * 12 / freq)
n += 1  # n is now number of full coupons since previous coupon
prev_coupon = edate(mat, -n * 12 / freq)

discounting_fraction = actual360.year_fraction(settle, next_coupon) * freq
accrual_fraction = actual360.year_fraction(prev_coupon, settle) * freq

if accrual_fraction == 1:
    # We are on coupon date. Assume that bond is ex-interest
    # Remove today's coupon
    discounting_fraction += 1
    accrual_fraction -= 1

print(f"""{n = },
{discounting_fraction = },
{accrual_fraction = },
{next_coupon = },
{prev_coupon = }
""")

# Time to Reciept of Cash Flows
cf_t = arange(start=(1 - accrual_fraction), stop=n, step=1)

# Bond Cash Flows
cf = np.full(n, coupon/2 * par)
cf[n-1] += par

# Present Value of Cash Flows
pv_cf = cf/((1+ytm/freq)**cf_t)

# Weight of Present Value of Cash Flows
cf_sum = np.sum(pv_cf)
weight = pv_cf / cf_sum
time_weighted = weight * cf_t

# Output Table
table = PrettyTable()
table.add_column("Period", arange(1,n+1))
table.add_column("Time to Receipt", np.round(cf_t, 4))
table.add_column("Cash Flow", cf)
table.add_column("PV", np.round(pv_cf, 4))
table.add_column("Weight", np.round(weight, 4))
table.add_column("Time Weighted", np.round(time_weighted, 4))
print(table)

macdur = np.sum(time_weighted) / freq
moddur = macdur / (1 + ytm / freq)

print(f'{macdur = }')
print(f'{moddur = }')

print(duration(cf=cf, rate=ytm, cf_t=cf_t, cf_freq=2, comp_freq=2))

n = 7,
discounting_fraction = 0.7,
accrual_fraction = 0.31666666666666665,
next_coupon = Timestamp('2027-10-15 00:00:00'),
prev_coupon = Timestamp('2027-04-15 00:00:00')

+--------+-----------------+-----------+---------+--------+---------------+
| Period | Time to Receipt | Cash Flow |    PV   | Weight | Time Weighted |
+--------+-----------------+-----------+---------+--------+---------------+
|   1    |      0.6833     |    1.6    |  1.5827 | 0.0157 |     0.0108    |
|   2    |      1.6833     |    1.6    |  1.5578 | 0.0155 |     0.0261    |
|   3    |      2.6833     |    1.6    |  1.5333 | 0.0153 |     0.0409    |
|   4    |      3.6833     |    1.6    |  1.5091 | 0.015  |     0.0553    |
|   5    |      4.6833     |    1.6    |  1.4854 | 0.0148 |     0.0692    |
|   6    |      5.6833     |    1.6    |  1.462  | 0.0145 |     0.0827    |
|   7    |      6.6833     |   101.6   | 91.3736 | 0.9092 |     6.0762    |
+--------+-----------------+-----------+---------+--------+----------

### Modified Duration

In [128]:
print(bond_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, modified=True, daycount=daycount))

3.1304966933630394
